In [1]:
import pandas as pd
import scipy.optimize as opt
import numpy as np

In [2]:
# read in demand data
demand = pd.read_csv("/content/citrus_demand.csv")

# read in supply predictions data
actual_supply = pd.read_csv("/content/citrus_quadratic_predict.csv")

# read in prices data
wholesale_prices = pd.read_csv("/content/wholesale_prices_short.csv")
retail_prices = pd.read_csv("/content/est_retail_prices.csv")

In [3]:
demand = demand[demand["year"] >= 1998]
demand = demand[demand["year"] <= 2015]
demand.reset_index(inplace=True, drop=True)

demand.head()

,year,Chicago,Detroit,Minneapolis.StPaul,Boston,New.York,Philadelphia,Atlanta,Baltimore,Dallas.FortWorth,Houston,Miami,DC,Los.Angeles,San.Francisco,San.Diego,Seattle,Average
0,1998,1.870383e+08,8.603941e+07,6.388088e+07,7.978738e+07,4.370930e+08,8.868379e+07,5.992135e+07,4.446037e+07,9.843968e+07,7.708341e+07,8.109145e+07,1.089691e+08,2.297290e+08,1.290025e+08,5.881579e+07,6.936250e+07,1.117293e+08
1,1999,1.825194e+08,8.914297e+07,6.514830e+07,9.450791e+07,4.323608e+08,1.055312e+08,6.781170e+07,5.212063e+07,1.193869e+08,8.806082e+07,9.633104e+07,1.129938e+08,2.517448e+08,1.327160e+08,6.042776e+07,7.911058e+07,1.194067e+08
2,2000,1.671226e+08,9.295417e+07,6.957600e+07,9.300697e+07,4.918061e+08,1.252924e+08,7.166764e+07,5.099217e+07,1.368396e+08,9.114355e+07,1.138096e+08,1.190512e+08,2.679443e+08,1.326530e+08,5.406173e+07,9.445342e+07,1.277867e+08
3,2001,1.796239e+08,8.332047e+07,6.673847e+07,9.114897e+07,4.670425e+08,1.194820e+08,6.486827e+07,4.562551e+07,1.306799e+08,8.265417e+07,1.097253e+08,1.024777e+08,2.389385e+08,1.157599e+08,4.523408e+07,8.862140e+07,1.195260e+08
4,2002,1.741256e+08,7.768032e+07,6.735025e+07,9.495969e+07,4.347284e+08,9.585756e+07,7.335671e+07,4.983372e+07,1.160534e+08,8.097091e+07,1.056303e+08,1.059825e+08,2.447750e+08,1.046570e+08,4.883273e+07,8.052063e+07,1.150185e+08


In [4]:
actual_supply = actual_supply[actual_supply["year"] >= 1999]
actual_supply.reset_index(inplace=True, drop=True)

actual_supply.head()

,year,west_lbs,p_sw_lbs,mw_se_lbs,a_ne_lbs,west_half_lbs,east_half_lbs
0,1999,5.607182e+08,6.130679e+07,1.466984e+09,0,6.220250e+08,1.466984e+09
1,2000,2.731724e+08,2.496791e+07,6.801667e+08,0,2.981403e+08,6.801667e+08
2,2001,7.803480e+08,8.609891e+07,2.015576e+09,0,8.664469e+08,2.015576e+09
3,2002,6.763036e+08,7.383637e+07,1.743261e+09,0,7.501400e+08,1.743261e+09
4,2003,6.314662e+08,6.631019e+07,1.578303e+09,0,6.977764e+08,1.578303e+09


In [5]:
wholesale_prices = wholesale_prices[wholesale_prices["year"] >= 1999]
wholesale_prices = wholesale_prices[wholesale_prices["year"] <= 2015]
wholesale_prices.reset_index(inplace=True, drop=True)

wholesale_prices.head()

,year,Apples,Bananas,Grapes,PeachesNectarines,Strawberries,OtherBerries,Citrus,Melons,OtherFruit,...,Carrots,Broccoli,Spinach,Lettuce,StringBeans,Cabbage,Onions,Corn,OtherVeg,Potatoes
0,1999,21.3,35.0,33.00,28.6,74.4,102,17.250,11.101633,67.161655,...,16.8,24.1,32.1,15.098916,46.5,11.3,9.7,16.9,27.428610,6.9
1,2000,17.8,36.0,32.35,27.6,64.9,115,7.000,11.035726,79.001992,...,13.1,31.2,31.8,19.228349,42.6,12.3,11.2,18.5,28.129329,5.3
2,2001,22.9,38.0,34.50,30.7,75.8,109,9.625,11.863701,72.454395,...,17.1,26.5,32.4,19.287165,45.0,13.4,10.7,19.5,27.847922,10.8
3,2002,25.8,43.0,34.30,30.6,71.3,117,11.250,12.184179,87.104935,...,19.1,31.4,34.2,23.559195,47.6,12.9,12.1,19.2,27.546230,9.6
4,2003,26.1,41.0,34.45,29.1,74.9,120,8.125,12.388878,79.634487,...,19.0,32.7,36.9,21.827844,49.2,13.3,13.7,19.2,28.384431,7.3


In [6]:
retail_prices = retail_prices[retail_prices["year"] >= 1999]
retail_prices = retail_prices[retail_prices["year"] <= 2015]
retail_prices.reset_index(inplace=True, drop=True)

retail_prices.head()

,year,Apples,Bananas,Grapes,PeachesNectarines,Strawberries,OtherBerries,Citrus,Melons,OtherFruit,...,Carrots,Broccoli,Spinach,Lettuce,StringBeans,Cabbage,Onions,Corn,OtherVeg,Potatoes
0,1999,86.0,52.5,49.500,42.90,208,153.0,25.8750,16.652449,100.742482,...,25.20,36.15,48.15,22.648374,69.75,16.95,14.55,25.35,41.142914,10.35
1,2000,88.0,54.0,48.525,41.40,188,172.5,10.5000,16.553589,118.502987,...,19.65,46.80,47.70,28.842524,63.90,18.45,16.80,27.75,42.193994,7.95
2,2001,83.0,57.0,51.750,46.05,229,163.5,14.4375,17.795551,108.681593,...,25.65,39.75,48.60,28.930747,67.50,20.10,16.05,29.25,41.771883,16.20
3,2002,91.0,64.5,51.450,45.90,228,175.5,16.8750,18.276268,130.657402,...,28.65,47.10,51.30,35.338792,71.40,19.35,18.15,28.80,41.319345,14.40
4,2003,94.0,61.5,51.675,43.65,237,180.0,12.1875,18.583316,119.451730,...,28.50,49.05,55.35,32.741766,73.80,19.95,20.55,28.80,42.576646,10.95


In [7]:
# parameters
N = 16 # number of retailers
K = demand.shape[0]-1 # number of iterations (0th one is the initial condition)

# dataframes to keep track of orders
# columns represent retailers, rows represent iterations
orders_west = pd.DataFrame(columns=range(0,N), index=range(0,K+1))
orders_p_sw = pd.DataFrame(columns=range(0,N), index=range(0,K+1))
orders_mw_se = pd.DataFrame(columns=range(0,N), index=range(0,K+1))

# initial orders
for i in range(0,N):
  orders_west.iloc[0,i] = demand["Average"].iloc[0] / 3
  orders_p_sw.iloc[0,i] = demand["Average"].iloc[0] / 3
  orders_mw_se.iloc[0,i] = demand["Average"].iloc[0] / 3

In [8]:
def ret_cost_sim(q, k, curr_i, orders_west, orders_p_sw, orders_mw_se, demand, supply, wholesale_prices, retail_prices):
  Q_i = demand["Average"].iloc[k+1]

  total_request_west = q[0]
  total_request_west += orders_west.iloc[k,0:curr_i].sum() + orders_west.iloc[k,curr_i+1:].sum()

  total_request_p_sw = q[1]
  total_request_p_sw += orders_p_sw.iloc[k,0:curr_i].sum() + orders_p_sw.iloc[k,curr_i+1:].sum()

  total_request_mw_se = q[2]
  total_request_mw_se += orders_mw_se.iloc[k,0:curr_i].sum() + orders_mw_se.iloc[k,curr_i+1:].sum()

  if (total_request_west <= supply["west_lbs"][k]):
    curr_R_west = q[0]
  else:
    curr_R_west = supply["west_lbs"][k] * (q[0] / total_request_west)

  if (total_request_p_sw <= supply["p_sw_lbs"][k]):
    curr_R_p_sw = q[1]
  else:
    curr_R_p_sw = supply["p_sw_lbs"][k] * (q[1] / total_request_p_sw)

  if (total_request_mw_se <= supply["mw_se_lbs"][k]):
    curr_R_mw_se = q[2]
  else:
    curr_R_mw_se = supply["mw_se_lbs"][k] * (q[2] / total_request_mw_se)

  R = curr_R_west + curr_R_p_sw + curr_R_mw_se

  holding = max(R - Q_i, 0)
  stockout = max(Q_i - R, 0)

  holding = wholesale_prices["Citrus"][k] * holding
  stockout = retail_prices["Citrus"][k] * stockout

  cost = holding + stockout

  return cost

In [9]:
for k in range(0,K):
  print(k)
  curr_i = 0 # retailers are identical

  rranges = (slice(10**5, 10**6, 10**5), slice(10**5, 10**6, 10**5), slice(10**5, 10**6, 10**5))
  params_curr = (k, curr_i, orders_west, orders_p_sw, orders_mw_se, demand, actual_supply, wholesale_prices, retail_prices)
  opt_sol = opt.brute(ret_cost_sim, rranges, args=params_curr, full_output=True)

  for i in range(0,N):
    if opt_sol[0][0] > 0:
      orders_west.iloc[k+1,i] = opt_sol[0][0]
    else:
      orders_west.iloc[k+1,i] = 0

    if opt_sol[0][1] > 0:
      orders_p_sw.iloc[k+1,i] = opt_sol[0][1]
    else:
      orders_p_sw.iloc[k+1,i] = 0

    if opt_sol[0][2] > 0:
      orders_mw_se.iloc[k+1,i] = opt_sol[0][2]
    else:
      orders_mw_se.iloc[k+1,i] = 0

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [ ]:
actual_supply

,year,west_lbs,p_sw_lbs,mw_se_lbs,a_ne_lbs,west_half_lbs,east_half_lbs
0,1999,4.169453e+08,4.313735e+07,1.073575e+09,0,4.600827e+08,1.073575e+09
1,2000,4.901552e+08,5.140139e+07,1.256439e+09,0,5.415566e+08,1.256439e+09
2,2001,5.086947e+08,5.347956e+07,1.301032e+09,0,5.621742e+08,1.301032e+09
3,2002,5.207370e+08,5.362758e+07,1.304117e+09,0,5.743646e+08,1.304117e+09
4,2003,5.203682e+08,5.150852e+07,1.288845e+09,0,5.718767e+08,1.288845e+09
5,2004,4.956877e+08,5.068268e+07,1.201347e+09,0,5.463704e+08,1.201347e+09
6,2005,4.829181e+08,5.003288e+07,1.123082e+09,0,5.329510e+08,1.123082e+09
7,2006,5.087178e+08,5.321071e+07,1.073570e+09,0,5.619285e+08,1.073570e+09
8,2007,4.095807e+08,4.298944e+07,8.397900e+08,0,4.525701e+08,8.397900e+08
9,2008,4.779587e+08,5.029183e+07,9.661620e+08,0,5.282505e+08,9.661620e+08


In [10]:
orders_total = orders_west + orders_p_sw + orders_mw_se
(orders_total.iloc[:,0] - demand["Average"]) / 10**7

0           0.0
1      0.939827
2     70.137457
3      0.759581
4      2.492431
5      0.139421
6      0.443968
7      0.244244
8      6.194542
9      0.118844
10    11.125571
11     4.911777
12    20.516937
13     2.458804
14     12.55718
15     1.972785
16     4.740061
17     0.149211
dtype: object

In [ ]:
actual_supply["west_lbs"] + actual_supply["p_sw_lbs"] + actual_supply["mw_se_lbs"] - (16 * demand["Average"])

0     3.013401e+08
1    -9.322009e+08
2    -5.287130e+08
3    -1.947157e+08
4    -3.527838e+07
5    -1.275103e+08
6    -9.999997e+07
7    -6.268596e+08
8    -6.514562e+08
9    -4.493537e+08
10   -3.799062e+08
11   -4.394427e+08
12   -1.604045e+08
13   -3.422766e+08
14   -3.400628e+08
15    4.494257e+06
16   -2.517116e+08
17             NaN
dtype: float64

In [ ]:
orders_p_sw

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,37243097.689842,37243097.689842,37243097.689842,37243097.689842,37243097.689842,37243097.689842,37243097.689842,37243097.689842,37243097.689842,37243097.689842,37243097.689842,37243097.689842,37243097.689842,37243097.689842,37243097.689842,37243097.689842
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,544858114.050978,544858114.050978,544858114.050978,544858114.050978,544858114.050978,544858114.050978,544858114.050978,544858114.050978,544858114.050978,544858114.050978,544858114.050978,544858114.050978,544858114.050978,544858114.050978,544858114.050978,544858114.050978
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,62606932.215602,62606932.215602,62606932.215602,62606932.215602,62606932.215602,62606932.215602,62606932.215602,62606932.215602,62606932.215602,62606932.215602,62606932.215602,62606932.215602,62606932.215602,62606932.215602,62606932.215602,62606932.215602
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,92208969.172891,92208969.172891,92208969.172891,92208969.172891,92208969.172891,92208969.172891,92208969.172891,92208969.172891,92208969.172891,92208969.172891,92208969.172891,92208969.172891,92208969.172891,92208969.172891,92208969.172891,92208969.172891
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,144783654.414385,144783654.414385,144783654.414385,144783654.414385,144783654.414385,144783654.414385,144783654.414385,144783654.414385,144783654.414385,144783654.414385,144783654.414385,144783654.414385,144783654.414385,144783654.414385,144783654.414385,144783654.414385
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
actual_supply["mw_se_lbs"].iloc[0] / 16 / 10**7

6.709845478301062

In [11]:
orders_west.to_csv("/content/citrus_quadratic_orders_west.csv", index=False)
orders_p_sw.to_csv("/content/citrus_quadratic_orders_p_sw.csv", index=False)
orders_mw_se.to_csv("/content/citrus_quadratic_orders_mw_se.csv", index=False)